<a href="https://colab.research.google.com/github/jacksklar/pLLPRosetteDetector/blob/master/object_detection_DDN_5_19_19_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neuromast Detector

This model uses tensorflow and the MobileNet v1 CNN to detect epthelial protoneuromasts in two color 2D images of the zebrafish lateral line primordium 

#Install packages and and imports

In [0]:
#install tensorflow
#!pip install tensorflow-gpu
%%capture 

#clone the object detection models from github
!git clone https://github.com/tensorflow/models.git

In [0]:
#install packages
%%capture 

!sudo apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install --user Cython
!pip install --user contextlib2
!pip install --user jupyter
!pip install 'prompt-toolkit==1.0.15'

#need to restart runtime to use this downgraded version of prompt-tookit.

In [0]:
%%capture

import os
os.chdir('/content/models/research/')
#compile protobuf
!protoc object_detection/protos/*.proto --python_out=.

#run the setup script to set up the object detection environment
os.getcwd()
!python3.6 setup.py build

In [0]:
import numpy as np
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import re
import ntpath
from google.colab import drive

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


drive.mount('/content/gdrive')

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')

  
#add libraries to python path
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim

#display matplotlib inline
%matplotlib inline

#clone the github repo
Importing the labelled data from github for training, and the data to use as a test case after training


In [0]:

#clone the repo including the data files and the py files into the models folder

os.chdir("/content/models/research/")

#(remove the current cloned directory if there is one)
!rm -r pLLPRosetteDetector 

!git clone https://github.com/jacksklar/pLLPRosetteDetector
  
#move the data directory from the cloned repo into the models directory
!mv /content/models/research/pLLPRosetteDetector/data /content/models/research/data/

#copy generate_tf_record py to the models folder
!cp /content/models/research/pLLPRosetteDetector/generate_tfrecord.py /content/models/research/generate_tfrecord.py

In [0]:
#make the tf record using the modified generate_tf_record.py script

!python generate_tfrecord.py --csv_input=/content/models/research/data/train_labels.csv  --output_path=/content/models/research/train.record
!python generate_tfrecord.py --csv_input=/content/models/research/data/test_labels.csv  --output_path=/content/models/research/test.record

#Setting up to train the pre-existing model


In [0]:
%%capture

#make a folder to put the record files in under the research folder and move the record files there
os.chdir("/content/models/research/")
!mkdir rosette_detect
!mv train.record test.record rosette_detect

#Grab the ssd_movilenet_v1 model
!wget "http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2018_01_28.tar.gz"

#unzip the files from the model
!tar -xvf ssd_mobilenet_v1_coco_2018_01_28.tar.gz

#copy all the model files into the rosette_detect folder
!cp /content/models/research/ssd_mobilenet_v1_coco_2018_01_28/model.ckpt.* rosette_detect

#get the appropriate config file for the model we're using, in this case the edited version from the cloned github repo, and copy it to this directory
!cp /content/models/research/pLLPRosetteDetector/ssd_mobilenet_v1_coco.config /content/models/research/rosette_detect

os.chdir("/content/models/research/rosette_detect")

In [0]:
#make the label map file

%%writefile labelmap.pbtxt 

item {
  id: 1
  name: 'nm'
}

In [0]:
!ls


In [0]:
# set up the model params


#path to save the model outputs in
model_dir = '/content/models/research/rosette_detect/training/'

# remove content in output model directory if there is preexisting model runs and make a new directory.
#!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

  

Can skip the next steps if we have a pre-frozen inference graph

In [0]:
#train the model. We're using 100,000 steps here for trainin

!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path='/content/models/research/rosette_detect/ssd_mobilenet_v1_coco.config' \
    --model_dir='/content/models/research/rosette_detect/training/' \
    --alsologtostderr \
    --num_train_steps=100000 \
    --num_eval_steps=5000

In [0]:
#check the training files

os.chdir("/content/models/research/rosette_detect/training")
!ls

In [0]:
#export the trained inference graph


os.chdir("/content/models/research/rosette_detect/training")

output_directory = '/content/models/research/rosette_detect/'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path='/content/models/research/rosette_detect/ssd_mobilenet_v1_coco.config' \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}



In [0]:
#fozen inference graph should now be in here

!ls /content/models/research/rosette_detect/

In [0]:
#download the trained file if you want to


from google.colab import files
files.download(pb_fname)
files.download(label_map_pbtxt_fname)


In [0]:
#or, download the inference graph from githib, pretrained on 100,000 steps

os.chdir('/content/models/research/rosette_detect/')

!wget 'https://github.com/jacksklar/pLLPRosetteDetector/blob/master/frozen_inference_graph.pb'

In [0]:
os.chdir("/content/gdrive/My Drive/final_test")
!ls

In [0]:

#inference test on final image folder from google drive


import os
import glob

output_directory = '/content/models/research/rosette_detect/'
pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")
label_map_pbtxt_fname = '/content/models/research/rosette_detect/labelmap.pbtxt'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '/content/models/research/rosette_detect/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
#PATH_TO_TEST_IMAGES_DIR =  "/content/models/research/pLLPRosetteDetector/test"
PATH_TO_TEST_IMAGES_DIR =  "/content/gdrive/My Drive/final_test"


#assert os.path.isfile(pb_fname)
#assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.*"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
print(TEST_IMAGE_PATHS)

In [0]:

from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

num_classes = 1

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict
    

for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    
    #lets look at the output dict to see whats going wrong
    #print(image_path)
    #print(output_dict)
    
    #save the output dict to a file as csv format
    
    
    
    my_filename = ntpath.basename(image_path)
    my_filenumber = my_filename[6:10]
    for x in range(4):
      if output_dict['detection_scores'][x] > 0.5:
        myString = my_filenumber + ',' + str(output_dict['detection_scores'][x]) + "," + str(output_dict['detection_boxes'][x][0]) + "," + str(output_dict['detection_boxes'][x][1]) + "," + str(output_dict['detection_boxes'][x][2]) + "," + str(output_dict['detection_boxes'][x][3])
        print(myString)
        with open('output.csv', 'a') as output_file:
          output_file.write(myString + '\n')
        
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        min_score_thresh=.2,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    #plt.figure(figsize=IMAGE_SIZE)
    #plt.imshow(image_np)

In [0]:

files.download('output.csv')
